In [2]:
# in this architecture there si an framework where there is Resoning+Acting at the work 

In [97]:
import os 
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import Tool
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import StateGraph
from langgraph.graph import START,END
from typing import List,Annotated,TypedDict,Sequence
from langchain_core.messages import BaseMessage,HumanMessage,SystemMessage,AIMessage
from langgraph.graph.message import add_messages



In [98]:
#This is web search 
docs=WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/").load()
splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
chunk=splitter.split_documents(docs)

In [99]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(
    model='sentence-transformers/all-MiniLM-L6-v2'
)

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 252.42it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [100]:
texts = [
    "LangGraph is a library for building stateful, multi-actor applications with LLMs.",
    "It extends LangChain to support cyclic graphs.",
]
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_texts(texts, embeddings)
retriever = vectorstore.as_retriever()


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 257.23it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [101]:
retriever.invoke("What are the autonomous agen")

[Document(id='23260543-12de-4458-93f2-4e7c0ebbd29e', metadata={}, page_content='It extends LangChain to support cyclic graphs.'),
 Document(id='1214c184-9f57-44cf-859f-d970127d0d52', metadata={}, page_content='LangGraph is a library for building stateful, multi-actor applications with LLMs.')]

In [102]:
def retriever_tool_function(query:str)->str:
    print('using RAGRtriever tool')
    docs=retriever.invoke(query)
    return "\n".join([doc.page_content for doc in docs])

In [103]:
retriever_tool=Tool(
    name="RAGRETRIEVER",
    description="use thir tool to fetch relevant knowledge base info",
    func=retriever_tool_function
)
retriever_tool
print(retriever_tool)

name='RAGRETRIEVER' description='use thir tool to fetch relevant knowledge base info' func=<function retriever_tool_function at 0x00000182D0B79580>


In [104]:
#Wikipedia Tool in making
wiki_tool=WikipediaAPIWrapper(api_wrapper=WikipediaAPIWrapper)
wiki_tool

WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\PARAS\\Downloads\\agentic-ai-langraph\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=4000)

In [105]:
from langchain_groq.chat_models import ChatGroq
llm=ChatGroq(model="openai/gpt-oss-120b")

In [110]:
from langgraph.prebuilt import create_react_agent
tool=[retriever_tool,wiki_tool]
react_node=create_react_agent(llm,tools=tool)
react_node

C:\Users\PARAS\AppData\Local\Temp\ipykernel_1428\3927086132.py:3: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  react_node=create_react_agent(llm,tools=tool)


ValueError: The first argument must be a string or a callable with a __name__ for tool decorator. Got <class 'langchain_community.utilities.wikipedia.WikipediaAPIWrapper'>

In [111]:
class AgentState(TypedDict):
    messages:Annotated[Sequence[BaseMessage],add_messages]
    

In [112]:
builder=StateGraph(AgentState)
builder.add_node("react_agent",react_node)
builder.set_entry_point("react_agent")
builder.add_edge("react_agent",END)
graph=builder.compile()
graph

NameError: name 'react_node' is not defined